In [1]:
# 导入工具包
import cv2
import numpy as np

In [4]:
# 定义参数
img_dir = '.\images\credit_card_01.png'
template_dir = '.\images\ocr_a_reference.png'
# 自定义图像读取方法
def myimshow(array):
    cv2.imshow(f'{array.shape}', array)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
# 读取模板并将其转为灰度图像后，二值化操作，将背景转为黑色
template = cv2.imread(template_dir)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
myimshow(template_gray)
template_bin = cv2.threshold(template_gray, 10, 255, cv2.THRESH_BINARY_INV)[1]
myimshow(template_bin)

In [9]:
# 获取模板轮廓(mode=只检测外轮廓，method=只保留终点坐标)
contours, hierarchy = cv2.findContours(template_bin, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
# 在原图上画轮廓的时候必须先拷贝一份出来，否则原图会被修改
template_copy_contours = cv2.drawContours(template.copy(), contours, contourIdx=-1, color=(0, 0, 255), thickness=2)
myimshow(template_copy_contours)

In [86]:
# 获取模板所有轮廓的外接矩形
template_bound_recs = [cv2.boundingRect(contour) for contour in contours]
template_bound_recs = sorted(template_bound_recs,key=lambda x:x[0])
template_copy_drawrec = template.copy()
# 在复制出来的模板图原图上绘制外接矩形框,并且得到数字与模板图像区域的映射关系
digits = {}
for idx, rec in enumerate(template_bound_recs):
    x, y, x1, y1 = rec[0], rec[1], (rec[0] + rec[2]), (rec[1] + rec[3])
    roi = template_gray[y:y1, x:x1]
    roi = cv2.resize(roi, (57, 88))
    digits[idx] = roi
    cv2.rectangle(template_copy_drawrec, (x, y), (x1, y1), color=(0, 0, 255), thickness=2)
    myimshow(roi)
    myimshow(template_copy_drawrec)
cv2.imwrite('./save_img/template_copy_drawrec.jpg',template_copy_drawrec)

True

In [82]:
# 二.对于待检测图像的处理
# 初始化卷积核
reckernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
sqkernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
# 读取图像与预处理
card = cv2.imread(img_dir)
myimshow(card)
# card = cv2.resize(card, (card.shape[1] * card.shape[0] // 300, 300), interpolation=cv2.INTER_AREA)
card_gray = cv2.cvtColor(card, cv2.COLOR_BGR2GRAY)
myimshow(card_gray)

In [83]:
# 礼帽操作(原始图像-闭运算)，突出暗背景下的高亮区域
tophat = cv2.morphologyEx(card_gray, op=cv2.MORPH_TOPHAT, kernel=reckernel)
myimshow(tophat)

# 计算梯度，得到边缘位置
gradx = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)

# 对梯度进行归一化
gradx = np.absolute(gradx)
minval, maxval = np.min(gradx), np.max(gradx)
gradx = (255 * (gradx - minval) / (maxval - minval)).astype(np.uint8)
myimshow(gradx)

In [87]:
# 通过闭操作将数字连在一起
close = cv2.morphologyEx(gradx, cv2.MORPH_CLOSE, reckernel)
myimshow(close)

# 通过THRESH_OTSU 自动寻找合适的阈值，适合双峰，需要把阈值参数设置为0
_, threshold = cv2.threshold(close, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
myimshow(threshold)

# 通过闭操作将数字连在一起
close = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, sqkernel, iterations=4)
myimshow(close)

cv2.imwrite('./save_img/close.jpg',close)

True

In [77]:
# 计算轮廓
contours_card, _ = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
card_contours = cv2.drawContours(card_gray.copy(), contours_card, -1, (255, 255, 255), 2)
myimshow(card_contours)
areas = []
for contour in contours_card:
    x, y, w, h = cv2.boundingRect(contour)
    ar = w / float(h)
    # 使用labelIMG获取ROI区域的长宽比以及具体信息
    if 3.0 < ar < 5.0 and ( 50 <  w < 160) and ( 10 < h < 60):
        areas.append((x, y, w, h))
# 按照X坐标顺序从左到右排序
card_gray_copy = card_gray.copy()
areas = sorted(areas, key=lambda x: x[0])

In [89]:
# 在灰度图中，便利每一个轮廓
card_copy = card.copy()
for idx, (x, y, w, h) in enumerate(areas) :
    groups = []
    group = card_gray[y - 5:y + h + 5, x - 5:x + w + 5]
    # group = cv2.threshold(group, 1, 255, cv2.THRESH_BINARY_INV)[1]
    # 对截取出来的部分，进行自适应阈值二值转换
    _, groups_auto = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # 在截取出来的区域再找到每个数字的轮廓
    contour_singles, _ = cv2.findContours(groups_auto, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    single_bounds = [cv2.boundingRect(contour_single) for contour_single in contour_singles]
    single_bounds = sorted(single_bounds,key=lambda x:x[0])
    # 便利每一个数字
    for single_bound in single_bounds:
        x1, y1, w1, h1 = single_bound
        roi = groups_auto[y1:y1 + h1, x1:x1 + w1]
        roi = cv2.resize(roi, (57, 88))
        roi = cv2.threshold(roi,10,255,cv2.THRESH_BINARY_INV)[1]
        scores = []
        for number, array in digits.items():
            res = cv2.matchTemplate(roi, array, cv2.TM_CCOEFF)
            score = cv2.minMaxLoc(res)[1]
            scores.append(score)
        groups.append(str(np.argmax(scores)))

    cv2.rectangle(card_copy, (x-5, y-5), (x + w+5, y + h+5), color=(0, 0, 255), thickness=2)
    cv2.putText(card_copy, "".join(groups), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    myimshow(card_copy)
cv2.imwrite('./save_img/final.jpg',card_copy)

True